In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
from typing import List, Protocol, runtime_checkable, Callable, ClassVar, Dict, Any
from abc import ABC, abstractmethod
from dataclasses import dataclass
# The attrs library is a powerful tool for creating robust classes.
# It needs to be installed: pip install attrs
import attrs
from attrs import define, field as attrs_field, validators
import numpy as np, pandas as pd, matplotlib.pyplot as plt

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams["figure.dpi"] = 130

# --- Utility Functions ---
def note(msg, **kwargs):
    """Prints a formatted message with a notebook icon."""
    formatted_msg = textwrap.fill(msg, width=100, subsequent_indent='   ')
    print(f"\n📝 {formatted_msg}", **kwargs)
def sec(title):
    """Prints a formatted section title for code blocks."""
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized.")

# Part 1: Foundations
## Chapter 1.11: A Paradigm for Managing Complexity: Object-Oriented Programming

### Introduction: The Challenge of Stateful Complexity

As computational models in economics—especially simulations like agent-based models (ABMs) or dynamic structural models—grow in scale, we face a fundamental challenge: managing **stateful complexity**. A purely procedural approach, consisting of functions operating on a collection of global data structures (e.g., lists of wealth, dictionaries of parameters), quickly becomes tangled. A change in one function can have unforeseen side effects on the shared state, making the system difficult to reason about, test, and extend. This is akin to having a large workshop where all tools and parts are thrown into a single pile; finding what you need and ensuring it works correctly becomes a nightmare.

**Object-Oriented Programming (OOP)** is a paradigm designed specifically to solve this problem. It organizes code by bundling data (**attributes**) and the behavior that operates on that data (**methods**) into self-contained **objects**. This principle, known as **encapsulation**, creates logical, independent components that communicate through well-defined public interfaces. It's like organizing the workshop into labeled toolboxes, each containing specific tools (methods) and the parts they work on (attributes).

For economists, OOP provides an intuitive and powerful way to structure models. We can represent a `Household` as an object with data like `wealth` and `labor_supply`, and methods like `consume()` and `save()`. A `Firm` can have `capital` and `technology` and a `produce()` method. A `Government` can have `tax_rate` and a `levy_taxes()` method. This approach allows us to build powerful abstractions that map directly onto the theoretical concepts we work with, leading to code that is more modular, robust, and intellectually coherent.

### 1. The Anatomy of a Class

A `class` is a blueprint for creating objects (also called instances). It defines the attributes and methods that all objects of that type will share.

#### 1.1 Constructor, Attributes, and Methods

- **`__init__` (The Constructor):** A special method called automatically when a new object is created. It initializes the object's state.
- **`self` (The Instance Reference):** The first parameter of every instance method, conventionally named `self`. It is a reference to the specific instance of the class, allowing the method to access and modify the instance's attributes.
- **Instance Attributes:** Data that is unique to each instance (e.g., `self.wealth`).
- **Class Attributes:** Data that is shared among *all* instances of a class (e.g., a universal parameter or a species name).
- **Instance Methods:** Functions defined inside a class that operate on an instance's data.

#### 1.2 Class and Static Methods

Beyond instance methods, Python provides two other types of methods that can be defined on a class, distinguished by decorators:

- **`@classmethod`**: This method receives the **class itself** as its first argument, conventionally named `cls`. It operates on the class, not the instance. Class methods are most commonly used for creating **alternative constructors**. For example, you might want to create a `Household` instance from a dictionary or a row in a DataFrame, rather than from individual arguments. A class method is the perfect tool for this.

- **`@staticmethod`**: This method does not receive any special first argument (neither `self` nor `cls`). It is essentially a regular function that is namespaced within the class. It cannot modify object state or class state. Static methods are used for utility functions that are logically related to the class but do not depend on the state of a specific instance.

In [ ]:
sec("A Household Class with Alternative Constructors")

class Household:
    """Represents a household in a simple economic model."""
    # A Class Attribute is shared by all instances of the class.
    COUNTRY_CODE: ClassVar[str] = "US"
    
    # The __init__ method is the constructor, run when a new instance is created.
    def __init__(self, hh_id: int, initial_wealth: float, initial_income: float):
        # Instance attributes are unique to each instance.
        self.id = hh_id
        self.wealth = float(initial_wealth)
        self.income = float(initial_income)
        
    # A classmethod receives the class `cls` as the first argument, not the instance `self`.
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Household':
        """An alternative constructor that creates a Household from a dictionary."""
        # `cls` here refers to the Household class itself.
        # This allows subclasses to reuse this constructor and create instances of their own type.
        return cls(hh_id=data['id'], initial_wealth=data['wealth'], initial_income=data['income'])
    
    # A staticmethod is like a regular function namespaced inside the class.
    # It doesn't receive the class or instance as an argument.
    @staticmethod
    def is_valid_id(hh_id: int) -> bool:
        """A utility function logically related to Households, but not dependent on state."""
        return isinstance(hh_id, int) and hh_id > 0

    # A regular instance method operates on the instance's state via `self`.
    def consume(self, amount: float):
        if amount > self.wealth: raise ValueError("Consumption cannot exceed wealth.")
        self.wealth -= amount
        
    # The __repr__ provides a developer-friendly string representation of the object.
    def __repr__(self) -> str:
        return f"Household(id={self.id}, wealth={self.wealth:.2f}, income={self.income:.2f})"

note("Creating an instance using the standard constructor:")
hh1 = Household(1, 1000, 200)
print(f"  {hh1}")

note("Creating an instance using the @classmethod alternative constructor:")
data_row = {'id': 2, 'wealth': 1500, 'income': 250}
hh2 = Household.from_dict(data_row)
print(f"  {hh2}")

note("Using the @staticmethod utility function:")
print(f"  Is ID 101 valid? {Household.is_valid_id(101)}")
print(f"  Is ID -5 valid? {Household.is_valid_id(-5)}")

### 2. The Pillars of OOP
The OOP paradigm is traditionally considered to rest on three conceptual pillars.

#### 2.1 Encapsulation and Information Hiding
**Encapsulation** is the core idea of bundling data and the methods that operate on that data into a single object. A key part of this is **information hiding**: the internal state of an object should be protected from arbitrary external modification. Instead, access should be controlled through a well-defined public interface (its methods).

Python does not have true `private` variables like Java or C++. Instead, it follows a philosophy of "we are all consenting adults here" and relies on naming conventions:
- **`_single_underscore`**: Signals that an attribute is for internal use. This is a convention only; it does not prevent access.
- **`__double_underscore`**: Triggers **name mangling**. An attribute like `__value` in class `MyClass` is renamed to `_MyClass__value`. This makes it harder (but not impossible) to access accidentally from outside, and is primarily used to avoid name clashes in inheritance.

**`@property`**: The Pythonic way to provide managed access to attributes is with the `@property` decorator. It allows you to expose what looks like a public attribute, but its access is controlled by getter, setter, and deleter methods. This is ideal for attributes that depend on other state or require validation when changed.

In [ ]:
sec("Encapsulation with @property")

class Asset:
    def __init__(self, initial_price: float, quantity: float):
        # The leading underscore signals these attributes are for internal use.
        self._price = initial_price
        self._quantity = quantity

    @property
    def value(self) -> float:
        """A read-only computed property. It has no setter, so it cannot be assigned to directly."""
        return self._price * self._quantity

    @property
    def price(self) -> float:
        """The getter for the price. This is called when you access `asset.price`."""
        return self._price

    @price.setter
    def price(self, new_price: float):
        """The setter for the price, with validation. This is called on `asset.price = ...`"""
        if new_price < 0:
            raise ValueError("Price cannot be negative.")
        self._price = new_price

stock = Asset(initial_price=100.0, quantity=50)
note("Accessing properties looks like accessing attributes:")
print(f"Initial price: {stock.price:.2f}")
print(f"Initial total value: {stock.value:.2f}")

note("\nUpdating the price via the property's setter:")
stock.price = 110.50 # This automatically calls the setter method.
print(f"New price: {stock.price:.2f}")
print(f"New total value: {stock.value:.2f}") # The 'value' property is recomputed automatically.

note("\nTrying to set an invalid price triggers the validation:")
try:
    stock.price = -5
except ValueError as e:
    print(f"  Caught expected error: {e}")

#### 2.2 Inheritance: Modeling 'is-a' Relationships
**Inheritance** allows a new class (the **subclass** or child) to be created as a specialized version of an existing class (the **superclass** or parent). The subclass inherits all attributes and methods of the superclass. It can then add new functionality or **override** inherited methods to provide a more specialized behavior. Inheritance models an **"is-a"** relationship: a `DSGEModel` *is a* type of `EconomicModel`.

The `super()` function is a crucial mechanism for building maintainable inheritance hierarchies. When you call `super().method()`, Python looks for `method` in the *next* class in the current class's **Method Resolution Order (MRO)**. This ensures that parent methods are called correctly and makes the code robust to changes in the inheritance structure (e.g., inserting a new base class into the hierarchy).

##### The Diamond Problem and the Method Resolution Order (MRO)
In languages that support **multiple inheritance**, an ambiguity known as the "diamond problem" can arise. Imagine a class `D` inherits from `B` and `C`, and both `B` and `C` inherit from a common ancestor `A`. If a method is called on an instance of `D` that is not defined in `D`, which parent's method should be used? Python solves this unambiguously using the **C3 linearization algorithm**, which computes a **Method Resolution Order (MRO)** for every class. You can inspect any class's MRO via its `__mro__` attribute or `.mro()` method.

![The Diamond Problem and MRO](../images/png/1.11-mro-diamond.png)

In [ ]:
sec("Inheritance in an Economic Context")

# The base class (or superclass)
class FinancialInstrument:
    def __init__(self, ticker: str, issue_date: str):
        self.ticker = ticker
        self.issue_date = issue_date
    
    def get_info(self) -> str:
        return f"Instrument {self.ticker}, issued {self.issue_date}"

# A subclass that inherits from FinancialInstrument
class Stock(FinancialInstrument):
    """A Stock 'is-a' FinancialInstrument."""
    def __init__(self, ticker: str, issue_date: str, sector: str):
        # `super()` calls the __init__ of the parent class (FinancialInstrument)
        # to handle the initialization of common attributes.
        super().__init__(ticker, issue_date)
        self.sector = sector # Add a new attribute specific to Stock
        
    # Override the parent's method to provide specialized behavior.
    def get_info(self) -> str:
        # It's good practice to call the parent's method to reuse its logic.
        base_info = super().get_info()
        return f"{base_info} [Type: Stock, Sector: {self.sector}]"

# Another subclass
class Bond(FinancialInstrument):
    """A Bond 'is-a' FinancialInstrument."""
    def __init__(self, ticker: str, issue_date: str, maturity_date: str, coupon_rate: float):
        super().__init__(ticker, issue_date)
        self.maturity_date = maturity_date
        self.coupon_rate = coupon_rate
        
    def get_info(self) -> str:
        base_info = super().get_info()
        return f"{base_info} [Type: Bond, Maturity: {self.maturity_date}]"

stock_a = Stock(ticker="AAPL", issue_date="1980-12-12", sector="Technology")
bond_b = Bond(ticker="T-BILL", issue_date="2023-01-01", maturity_date="2024-01-01", coupon_rate=0.05)

print(stock_a.get_info()) # Calls the overridden method in Stock
print(bond_b.get_info())   # Calls the overridden method in Bond

#### 2.3 Polymorphism and Defining Interfaces
**Polymorphism** (from Greek for "many forms") is the ability of different objects to respond to the same method call in different, type-specific ways. This allows for writing flexible code that can operate on a wide range of objects, making the system easier to extend.

In Python, there are two main ways to define a common interface that enables polymorphism:

1.  **Abstract Base Classes (ABCs):** An ABC (from the `abc` module) uses inheritance to define an interface. Subclasses must explicitly inherit from the ABC and implement its `@abstractmethod`s. This is checked at **runtime**; if a subclass fails to implement a required method, Python will raise a `TypeError` when you try to instantiate it. This is useful for frameworks where you want to strictly enforce that components adhere to a specific structure.

2.  **Protocols (Static Duck Typing):** A Protocol (from the `typing` module, introduced in Python 3.8) defines an interface based on **structure**, not inheritance. Any class that has the right set of methods and signatures is considered to implement the protocol, without needing to inherit from it. This is checked by **static type checkers** (like Mypy) before you even run the code. It is a more flexible, decoupled way of defining interfaces that aligns with Python's "duck typing" philosophy.

In [ ]:
sec("Defining Interfaces: ABC vs. Protocol")

# --- Approach 1: Abstract Base Class (Nominal Subtyping) ---
# Requires explicit inheritance.
class AbstractValuationModel(ABC):
    @abstractmethod
    def calculate_npv(self, cash_flows: List[float], discount_rate: float) -> float: ...

class DCFModel(AbstractValuationModel):
    """This class MUST implement calculate_npv to be instantiated."""
    def calculate_npv(self, cash_flows: List[float], discount_rate: float) -> float:
        return sum(cf / (1 + discount_rate)**t for t, cf in enumerate(cash_flows, 1))

# --- Approach 2: Protocol (Structural Subtyping) ---
# Defines an interface based on method names and signatures, no inheritance needed.
@runtime_checkable # Allows for runtime isinstance() checks, otherwise only static checkers see it.
class ValuationProtocol(Protocol):
    def calculate_npv(self, cash_flows: List[float], discount_rate: float) -> float: ...

class DividendDiscountModel: # Does NOT inherit from the protocol, but matches its structure.
    def calculate_npv(self, cash_flows: List[float], discount_rate: float) -> float:
        # Simplified DDM for demonstration
        return sum(cf / (1 + discount_rate)**t for t, cf in enumerate(cash_flows, 1))

# --- Polymorphic Function ---
# This function is polymorphic because it can accept any object that satisfies the ValuationProtocol.
def run_valuation(model: ValuationProtocol, cfs: List[float], r: float):
    # The same method call `model.calculate_npv` works on different types of objects.
    npv = model.calculate_npv(cfs, r)
    print(f"  -> Running {type(model).__name__}: NPV = ${npv:.2f}")

dcf = DCFModel()
ddm = DividendDiscountModel()
cash_flows = [10, 10, 10, 110]
rate = 0.1

note("Polymorphism in action: run_valuation works on both types:")
run_valuation(dcf, cash_flows, rate)
run_valuation(ddm, cash_flows, rate)

### 3. Core Design Principle: Prefer Composition Over Inheritance

While inheritance is powerful, it creates a tight coupling between the parent and child classes. Changes to a base class can easily break subclasses in unexpected ways (the **fragile base class problem**). A core principle of modern OOP is to **prefer composition over inheritance** as a tool for code reuse and building complex objects.

- **Inheritance (is-a):** A `Student` *is a* `Person`. This implies a strong, hierarchical relationship.
- **Composition (has-a):** A `Car` *has an* `Engine`. The `Car` object is *composed* of other, smaller, independent objects. This is a more flexible relationship.

Composition involves assembling complex objects from other, smaller, independent objects. This leads to more flexible, modular, and maintainable code, as you can easily swap out components. The **Strategy Pattern** is a formal application of this principle, where a "context" class is configured with a "strategy" object and delegates work to it, allowing the algorithm to be changed dynamically.

![Composition vs. Inheritance](../images/png/1.11-composition-vs-inheritance.png)

In [ ]:
sec("Building a Model via Composition")

# --- 1. Define Component Interfaces (Protocols are great for this) ---
class ProductionFunction(Protocol):
    def produce(self, K: float, L: float) -> float: ...
class UtilityFunction(Protocol):
    def calculate_utility(self, C: float) -> float: ...

# --- 2. Implement Concrete Components ---
class CobbDouglas:
    def __init__(self, alpha: float): self.alpha = alpha
    def produce(self, K: float, L: float) -> float: return K**self.alpha * L**(1 - self.alpha)

class CRRA:
    def __init__(self, gamma: float): self.gamma = gamma
    def calculate_utility(self, C: float) -> float: return (C**(1 - self.gamma)) / (1 - self.gamma)

# --- 3. Compose the Main Model Object ---
class DynamicModel:
    """This model 'has-a' production function and 'has-a' utility function."""
    def __init__(self, prod_func: ProductionFunction, util_func: UtilityFunction):
        # The model is composed of other objects, which are stored as attributes.
        self.production = prod_func
        self.utility = util_func
        self.K = 100
        self.L = 50
        
    def step(self):
        """Simulate one step of the model by delegating work to its components."""
        Y = self.production.produce(self.K, self.L)
        # Assume simple consumption/investment rule
        C = 0.7 * Y
        I = 0.3 * Y
        self.K += I - (0.05 * self.K) # Investment adds to capital, less depreciation
        U = self.utility.calculate_utility(C)
        print(f"  Output={Y:.2f}, Consumption={C:.2f}, Utility={U:.2f}, New K={self.K:.2f}")

note("Creating a model by composing different components:")
# We can easily swap out components to create a different model configuration.
model = DynamicModel(
    prod_func=CobbDouglas(alpha=0.33),
    util_func=CRRA(gamma=2.0)
)
model.step()
model.step()

### 4. Tools for Better Classes: `dataclasses` and `attrs`
Writing the boilerplate methods `__init__`, `__repr__`, `__eq__`, etc., for every class is tedious and error-prone. Modern Python provides tools to automate this.

- **`@dataclasses.dataclass`**: A standard library decorator (Python 3.7+) that auto-generates these methods based on type-annotated class attributes. It is excellent for creating simple, mutable data containers.

- **`attrs`**: A powerful third-party library that offers everything `dataclasses` does plus advanced features like **validators** (for checking correctness) and **converters** (for normalizing data). It is the superior choice for creating complex, self-validating objects that are the bedrock of robust models.

In [ ]:
sec("Comparing `dataclasses` and `attrs`")

@dataclass(frozen=True)
class DataClassParams:
    """A simple, immutable data container using the standard library."""
    alpha: float
    beta: float

# `define` is the main decorator from attrs.
# `frozen=True` makes instances immutable after creation.
# `slots=True` is a performance optimization that reduces memory usage.
@define(frozen=True, slots=True)
class AttrsParams:
    """A robust, immutable, self-validating container with attrs."""
    # `attrs_field` allows for adding metadata like validators.
    alpha: float = attrs_field(validator=validators.instance_of(float))
    beta: float = attrs_field(validator=[validators.ge(0), validators.lt(1)])

dc = DataClassParams(alpha=0.3, beta=0.9)
ap = AttrsParams(alpha=0.3, beta=0.9)

print(f"Dataclass instance: {dc}") # Note the nice __repr__ is auto-generated
print(f"Attrs instance:     {ap}") # Attrs also provides a nice __repr__

note("Attrs provides validation that dataclasses do not:")
try:
    # This will fail because beta=1.1 violates the validator `validators.lt(1)`
    AttrsParams(alpha=0.3, beta=1.1)
except ValueError as e:
    print(f"  Caught expected attrs validation error: {e}")

### 5. A Note on Performance: OOP vs. Data-Oriented Design

While OOP is a powerful paradigm for managing logical complexity, it is not always the most performant approach for heavy numerical computation. The standard OOP approach involves creating many individual objects (e.g., a list of 1,000,000 `Household` objects). Iterating over this list in Python can be slow because each object is a separate entity in memory, leading to poor cache locality.

**Data-Oriented Design (DOD)** offers an alternative. Instead of a list of objects, you maintain a few large, contiguous arrays of data. For example, you would have one NumPy array for `wealth`, one for `income`, etc., where `wealth[i]` corresponds to the wealth of agent `i`.

```python
# OOP Style
total_wealth = sum(agent.wealth for agent in agents)

# DOD Style
total_wealth = np.sum(agent_data['wealth'])
```

The DOD approach allows libraries like NumPy, Numba, and JAX to perform highly optimized, vectorized operations on the entire dataset at once, which is often orders of magnitude faster than a Python loop over objects.

**The takeaway is to choose the right tool for the job:**
- Use **OOP** to structure the high-level logic of your model, define components, manage configurations, and orchestrate the simulation.
- Use **Data-Oriented Design** within performance-critical kernels of your model where you are performing repetitive numerical calculations on large datasets.

### 6. Exercises

1.  **Alternative Constructor with `@classmethod`**
    - **Task:** Add a class method to the `Asset` class from Section 2.1 called `from_market_cap`. This method should accept `ticker`, `market_cap`, and `shares_outstanding` and should calculate the initial price internally to create and return a new `Asset` instance. Demonstrate its use.

2.  **OOP Design: "is-a" vs. "has-a"**: You are modeling a `Portfolio` which contains a collection of `Asset` objects. Should `Portfolio` inherit from `list`? Or should it be a separate object that *has* a list of assets as an attribute (e.g., `self.assets = []`)? Justify your choice using the "is-a" vs. "has-a" principle and explain why the alternative is a poor design. (Hint: Consider what would happen if someone called the `.sort()` method on your `Portfolio` if it inherited from `list`—what would that imply about the portfolio itself?)

3.  **`attrs` with Validators:** Create an `attrs` class `AgentPreferences` with two attributes: `beta` (discount factor) and `gamma` (risk aversion). Use `attrs` validators to ensure that `beta` is always between 0 and 1 (inclusive of 0, exclusive of 1) and `gamma` is always positive. Demonstrate that creating an instance with invalid parameters raises a `ValueError`.

4.  **The Strategy Pattern for Consumption:** A `Household` agent needs to decide how much to consume out of their wealth. This decision can follow different rules (strategies).
    - **Task:** Define a `ConsumptionStrategy` protocol with a method `decide_consumption(wealth: float) -> float`. Implement two strategies: `FixedFractionStrategy` which consumes a fixed fraction of wealth (e.g., 20%), and `TargetWealthStrategy` which consumes wealth above a certain target level.
    - Create a `Household` class that is initialized with a consumption strategy object. Demonstrate that you can create two households with different strategies and that they behave differently when you call a `consume()` method on them.

5.  **Combining Composition and Protocols:** Create an `attrs` class named `Market` that is composed of a `list` of `Household` objects and a `PricingModel` object. Define a `PricingModel` protocol with a method `find_equilibrium_price(households: List[Household]) -> float`. Implement a simple `AverageDemandPricer` that fulfills this protocol. The `Market` class should have a method `clear_market()` that uses its pricing model to calculate and print the equilibrium price.